In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import os

# a) Load the dataset
# Assuming the BCCD dataset is structured with directories for each class (WBC, RBC, Platelets)
from google.colab import drive
drive.mount('/content/drive')

base_dir = '/content/drive/MyDrive/BCCD'  # Update with actual dataset path
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# b) Show the number of testing and training images
def count_images(directory):
    total_images = 0
    for subdir in os.listdir(directory):
        total_images += len(os.listdir(os.path.join(directory, subdir)))
    return total_images

train_size = count_images(train_dir)
validation_size = count_images(validation_dir)
print(f"Number of training images: {train_size}")
print(f"Number of testing images: {validation_size}")

# c) Plot some images
def plot_samples(directory, num_images=9):
    plt.figure(figsize=(10, 10))
    i = 0
    for subdir in os.listdir(directory):
        for img_name in os.listdir(os.path.join(directory, subdir))[:num_images // 3]:
            img = tf.keras.preprocessing.image.load_img(os.path.join(directory, subdir, img_name), target_size=(150, 150))
            plt.subplot(3, 3, i + 1)
            plt.imshow(img)
            plt.axis('off')
            i += 1
    plt.suptitle('Sample Images from Dataset')
    plt.show()

plot_samples(train_dir)

# d) Do image augmentation – contrast, flipping, and rotation
train_datagen_augmented = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2]
)

validation_datagen = ImageDataGenerator(rescale=1.0/255)

# Load images with augmentation
train_generator_augmented = train_datagen_augmented.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# e) After augmentation, show the number of testing and training images
print(f"Number of training images (augmented): {train_size}")
print(f"Number of testing images: {validation_size}")

# f) Normalizing the training data (already done in ImageDataGenerator with rescale=1.0/255)

# g) Build a CNN to train images without augmentation
cnn_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')  # Three classes: WBC, RBC, Platelets
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train CNN without augmentation
history_no_aug = cnn_model.fit(
    validation_generator,  # Without augmentation
    epochs=5,
    validation_data=validation_generator
)

# h) Show training/testing accuracy without augmentation
plt.plot(history_no_aug.history['accuracy'], label='Training Accuracy (No Augmentation)')
plt.plot(history_no_aug.history['val_accuracy'], label='Validation Accuracy (No Augmentation)')
plt.legend()
plt.title("Training and Validation Accuracy without Augmentation")
plt.show()

# i) Build a CNN to train images with augmentation
cnn_model_aug = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')  # Three classes
])

cnn_model_aug.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train CNN with augmentation
history_aug = cnn_model_aug.fit(
    train_generator_augmented,  # With augmentation
    epochs=5,
    validation_data=validation_generator
)

# j) Show training/testing accuracy with augmentation
plt.plot(history_aug.history['accuracy'], label='Training Accuracy (With Augmentation)')
plt.plot(history_aug.history['val_accuracy'], label='Validation Accuracy (With Augmentation)')
plt.legend()
plt.title("Training and Validation Accuracy with Augmentation")
plt.show()

# k) Compare training/testing accuracy before and after augmentation
plt.plot(history_no_aug.history['accuracy'], label='Training Accuracy (No Augmentation)')
plt.plot(history_no_aug.history['val_accuracy'], label='Validation Accuracy (No Augmentation)')
plt.plot(history_aug.history['accuracy'], label='Training Accuracy (With Augmentation)')
plt.plot(history_aug.history['val_accuracy'], label='Validation Accuracy (With Augmentation)')
plt.legend()
plt.title("Comparison of Model Accuracy with and without Augmentation")
plt.show()
